<a href="https://colab.research.google.com/github/oteri/protein_sequence_classifier/blob/main/ESM2_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/oteri/protein_sequence_classifier.git

Cloning into 'protein_sequence_classifier'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 77 (delta 31), reused 65 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 194.07 KiB | 12.13 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!cd protein_sequence_classifier/ && uv pip install . && make download_all_datasets

Using Python 3.12.12 environment at: /usr
Resolved 111 packages in 1.28s
Prepared 7 packages in 3.31s
Installed 7 packages in 209ms
 + biopython==1.86
 + jedi==0.19.2
 + loro==1.10.3
 + marimo==0.18.4
 + msgspec==0.20.0
 + protein-sequence-classifier==0.1.0 (from file:///content/protein_sequence_classifier)
 + pymdown-extensions==10.19.1
--2025-12-18 07:12:46--  https://raw.githubusercontent.com/LUCA-BioTech/AIL-scan/main/datasets/test/Cas1.faa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199573 (195K) [text/plain]
Saving to: ‘data/test/Cas1.faa’

data/test/Cas1.faa  100%[===================>] 194.90K  --.-KB/s    in 0.05s   

2025-12-18 07:12:46 (3.51 MB/s) - ‘data/test/Cas1.faa’ saved [199573/199573]

--2025-12-18 07:12:46--  https://raw.githubuserconte

In [17]:
from pathlib import Path
from datetime import datetime
import yaml
import torch
import logging
import argparse
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from peft import get_peft_model, LoraConfig, TaskType
from accelerate import Accelerator
from datasets import Dataset
from Bio import SeqIO
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    force=True,   # <-- Forcing reconfiguration so to be active even in colab
)
logger = logging.getLogger(__name__)


In [6]:
def load_config(config_path="configs/train.yaml"):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config

def load_data_from_folder(folder_path):
    sequences = []
    labels = []
    # Get all fasta files (support .fasta and .faa)
    folder = Path(folder_path)
    fasta_files = list(folder.glob("*.fasta")) + list(folder.glob("*.faa"))

    if not fasta_files:
        raise ValueError(f"No fasta/faa files found in {folder_path}")

    for file_path in fasta_files:
        # Extract label from filename (e.g., "Cas12.fasta" -> "Cas12")
        label = file_path.stem

        # Read sequences
        try:
            records = list(SeqIO.parse(file_path, "fasta"))
            if not records:
                logger.warning(f"No sequences found in {file_path}")
                continue
            for record in records:
                sequences.append(str(record.seq))
                labels.append(label)
        except Exception as e:
            logger.error(f"Error reading {file_path}: {e}")

    return sequences, labels

def create_dataset(folder_path, label_to_id=None):
    if not Path(folder_path).exists():
        raise ValueError(f"Directory {folder_path} does not exist.")

    sequences, labels_text = load_data_from_folder(folder_path)

    if not sequences:
        raise ValueError(f"No sequences collected from {folder_path}")

    if label_to_id is None:
        unique_labels = sorted(list(set(labels_text)))
        label_to_id = {label: i for i, label in enumerate(unique_labels)}

    # Filter labels
    labels_id = []
    filtered_sequences = []
    for seq, label in zip(sequences, labels_text):
        if label in label_to_id:
            labels_id.append(label_to_id[label])
            filtered_sequences.append(seq)
        else:
            logger.warning(f"Label {label} not in label map. Skipping sequence.")

    from datasets import Dataset # Import here to ensure visibility if needed or rely on outer scope
    # Changed "label" to "labels" for HF model compatibility
    dataset = Dataset.from_dict({"sequence": filtered_sequences, "labels": labels_id})
    return dataset, label_to_id

In [7]:
# Simulate args
config_file = "protein_sequence_classifier/configs/train.yaml"

# 1. Initialize Accelerator
accelerator = Accelerator()
logger.info("Accelerator initialized")

# 2. Load Config
config = load_config(config_file)
logger.info(f"Loaded config: {config}")

Accelerator initialized
Loaded config: {'model_name': 'facebook/esm2_t6_8M_UR50D', 'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.1, 'batch_size': 8, 'max_length': 512, 'learning_rate': 0.0005, 'num_epochs': 3, 'output_dir': './output', 'seed': 42, 'label_smoothing': 0.1}


In [8]:
# 3. Load Data & Create Label Map
logger.info("Loading training data...")
train_path = "protein_sequence_classifier/data/train"
val_path  =  "protein_sequence_classifier/data/validate"
test_path  = "protein_sequence_classifier/data/test"

try:
  train_dataset, label_to_id = create_dataset()
except ValueError:
    raise ValueError(f"Training data is empty! Please populate {train_path}.")

id_to_label = {v: k for k, v in label_to_id.items()}
num_labels = len(label_to_id)
logger.info(f"Found {num_labels} classes: {label_to_id}")


logger.info(f"Loading validation and test data from {val_path} and {test_path}")

try:
  val_dataset, _ = create_dataset(val_path, label_to_id)
except ValueError:
  raise ValueError(f"Validation data is empty! Please populate {val_path}.")

try:
  test_dataset, _ = create_dataset(test_path, label_to_id)
except ValueError:
  raise ValueError(f"Test data is empty! Please populate {test_path}.")

Loading training data...
Found 13 classes: {'Cas1': 0, 'Cas10': 1, 'Cas12': 2, 'Cas13': 3, 'Cas2': 4, 'Cas3': 5, 'Cas4': 6, 'Cas5': 7, 'Cas6': 8, 'Cas7': 9, 'Cas8': 10, 'Cas9': 11, 'nocas': 12}
Loading validation and test data...


In [10]:
# 4. Tokenization
model_name = config["model_name"]
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_batch(batch):
    return tokenizer(batch["sequence"], padding="max_length", truncation=True, max_length=config["max_length"])

with accelerator.main_process_first():
    train_dataset_tokenized = train_dataset.map(tokenize_batch, batched=True, remove_columns=["sequence"])
    val_dataset_tokenized = val_dataset.map(tokenize_batch, batched=True, remove_columns=["sequence"])
    test_dataset_tokenized = test_dataset.map(tokenize_batch, batched=True, remove_columns=["sequence"])

train_dataset_tokenized.set_format("torch")
val_dataset_tokenized.set_format("torch")
test_dataset_tokenized.set_format("torch")

Map:   0%|          | 0/86855 [00:00<?, ? examples/s]

Map:   0%|          | 0/2759 [00:00<?, ? examples/s]

Map:   0%|          | 0/39694 [00:00<?, ? examples/s]

In [11]:
# 5. DataLoaders
batch_size = config["batch_size"]
train_dataloader = DataLoader(train_dataset_tokenized, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset_tokenized, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset_tokenized, batch_size=batch_size)

In [12]:
# 6. Model
logger.info(f"Loading model {model_name}...")
_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

# 7. LoRA
logger.info("Applying LoRA...")
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=config["lora_r"],
    lora_alpha=config["lora_alpha"],
    lora_dropout=config["lora_dropout"],
    target_modules=["query", "key", "value", "dense"]
)
model_lora = get_peft_model(_model, peft_config)
if accelerator.is_local_main_process:
    model_lora.print_trainable_parameters()

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 383,373 || all params: 7,899,347 || trainable%: 4.8532


In [13]:
# 8. Optimizer & Scheduler
optimizer = torch.optim.AdamW(model_lora.parameters(), lr=config["learning_rate"])

num_epochs = config["num_epochs"]
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [14]:
# 9. Prepare
# Use new variable names to avoid marimo global variable redefinition conflict
p_model, p_optimizer, p_train_dataloader, p_val_dataloader, p_lr_scheduler = accelerator.prepare(
    model_lora, optimizer, train_dataloader, val_dataloader, lr_scheduler
)
if test_dataloader:
    p_test_dataloader = accelerator.prepare(test_dataloader)
else:
    p_test_dataloader = None

In [18]:
# 10. Training Loop
def run_training_loop():
    logger.info("Starting training...")
    for epoch in range(num_epochs):
        p_model.train()
        total_loss = 0
        for i, batch in enumerate(p_train_dataloader):
            if i == 0:
                # Debug print
                if 'labels' not in batch:
                     logger.warn(f"WARNING: 'labels' key missing in batch! Keys: {list(batch.keys())}")

            outputs = p_model(**batch)

            loss = outputs.loss
            if loss is None:
                 raise ValueError(f"Model return None loss. Batch keys: {list(batch.keys())}")

            accelerator.backward(loss)
            p_optimizer.step()
            p_lr_scheduler.step()
            p_optimizer.zero_grad()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(p_train_dataloader)
        logger.info(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")


        # Validation
        if p_val_dataloader:
            p_model.eval()
            val_loss = 0
            all_preds = []
            all_labels = []
            for batch in p_val_dataloader:
                with torch.no_grad():
                    outputs = p_model(**batch)
                    val_loss += outputs.loss.item()
                    predictions = outputs.logits.argmax(dim=-1)
                    # Changed "label" to "labels"
                    preds, labels = accelerator.gather_for_metrics((predictions, batch["labels"]))
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            avg_val_loss = val_loss / len(p_val_dataloader)
            accuracy = accuracy_score(all_labels, all_preds)
            logger.info(f"Epoch {epoch+1}/{num_epochs} - Val Loss: {avg_val_loss:.4f} - Val Accuracy: {accuracy:.4f}")

run_training_loop()

2025-12-18 08:05:28,188 - INFO - __main__ - Starting training...
2025-12-18 08:35:36,353 - INFO - __main__ - Epoch 1/3 - Train Loss: 0.0503
2025-12-18 08:36:00,058 - INFO - __main__ - Epoch 1/3 - Val Loss: 0.1944 - Val Accuracy: 0.9652
2025-12-18 09:05:55,105 - INFO - __main__ - Epoch 2/3 - Train Loss: 0.0234
2025-12-18 09:06:14,758 - INFO - __main__ - Epoch 2/3 - Val Loss: 0.1565 - Val Accuracy: 0.9728
2025-12-18 09:35:57,085 - INFO - __main__ - Epoch 3/3 - Train Loss: 0.0100
2025-12-18 09:36:16,788 - INFO - __main__ - Epoch 3/3 - Val Loss: 0.1575 - Val Accuracy: 0.9750


In [ ]:
# 11. Final Evaluation on Test Set
def run_test_loop():
    if p_test_dataloader:
        logger.info("Evaluating on test set...")
        p_model.eval()
        all_preds = []
        all_labels = []
        for batch in p_test_dataloader:
            with torch.no_grad():
                outputs = p_model(**batch)
                predictions = outputs.logits.argmax(dim=-1)
                # Changed "label" to "labels"
                preds, labels = accelerator.gather_for_metrics((predictions, batch["labels"]))
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)

        logger.info(f"Test Results: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

run_test_loop()

In [ ]:
# 12. Save Model
output_dir = config["output_dir"]
if output_dir:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(p_model)
    unwrapped_model.save_pretrained(output_dir, is_main_process=accelerator.is_main_process)
    tokenizer.save_pretrained(output_dir)
    if accelerator.is_main_process:
        # Save label map
        with open(Path(output_dir) / "label_map.yaml", "w") as f:
            yaml.dump(label_to_id, f)
        logger.info(f"Model saved to {output_dir}")